<a href="https://colab.research.google.com/github/KaranJoseph/DemandForecasting_SCA/blob/main/Codes/Inventory_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pyomo and GLPK
!pip install -q pyomo
!apt-get install -y -qq glpk-utils #if GLPK is used
# !apt-get install -y -qq coinor-cbc #if cbc is used

     |████████████████████████████████| 9.6 MB 577 kB/s 
     |████████████████████████████████| 49 kB 2.7 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/KaranJoseph/DemandForecasting_SCA/main/Data/Output.csv")
df = df[["Item_ID", "Ship_Date", "Pred"]].dropna()

maxx = df["Pred"].max() + 50000 #Big M constraint

In [ ]:
# Assumptions

Co = 0.5 #Ordering Cost per unit (Just for the sake of scaling the order cost by unit)
Ch = 0.05 #Holding Cost per unit

In [ ]:
from pyomo.environ import *

def optimizer(x, Co):
  """
    Parameters
    ----------
    x : numpy array
        Forecasted shipment quantities for 8 weeks
    Co : float 
        Order cost multiplier

    Returns
    -------
    model : pyomo object 
            Optimized inventory management model based on the obj and constraints of shortest path method (Integer Linear Programming Model)
    Co : float
          Lot4Lot inventory calculation
  """
  Co = x.mean() * Co

  model = ConcreteModel()
  #Variables 
  model.y = Var(list(range(1, len(x)+1)), within=Binary) # Order yes or no
  model.s = Var(list(range(1, len(x)+1)), within=NonNegativeReals) # Inventory at period i
  model.q = Var(list(range(1, len(x)+1)), within=NonNegativeReals) # Order Quantity at period i

  #Objective
  obj1 = 0
  obj2 = 0
  for i in range(1, len(x)+1):
    obj1 += model.y[i]
    obj2 += model.s[i]
  model.OBJ = Objective(sense=minimize, expr = Co*obj1 + Ch*obj2)

  #Constraints
  model.order = ConstraintList()
  model.inv = ConstraintList()

  #Constraint 1: Constraint for order quantity
  for i in range(1, len(x)+1):
    const_expr = model.q[i] <= model.y[i]*maxx
    model.order.add(expr = const_expr)
  
  #Constraint 2: Constraint for quantity meeting expected demand
  model.inv.add(expr = model.s[1] - model.q[1] + x[0] == 0) #Initial Inventory is considered as 0
  for i in range(2, len(x)+1):
    const_expr = model.s[i] - model.s[i-1] - model.q[i] + x[i-1] == 0
    model.inv.add(expr = const_expr)

  #model.pprint()
  opt = SolverFactory('glpk')
  opt.solve(model) 
  #model.display()
  return model, Co*len(x)

In [ ]:
df_im = pd.DataFrame()
cost = {}
lot4lot = []

for item in df["Item_ID"].unique(): #Iterate through each item_id and calls optimizer function to get the best IM solution at Item level
  temp = df[df["Item_ID"] == item].reset_index().drop("index", axis=1)
  model, val = optimizer(temp["Pred"].values, Co)
  lot4lot.append(val)
  order = []
  qty = []
  inv = []
  for i in range(1, temp.shape[0]+1):
    order.append(model.y[i].value)
    qty.append(model.q[i].value)
    inv.append(model.s[i].value)

  qty = pd.Series(qty, name="OrderQty")
  inv = pd.Series(inv, name="Inventory")
  order = pd.Series(order, name="Order(yes/no)")
  t = pd.concat([temp, qty, inv, order], axis=1)
  df_im = df_im.append(t)

  cost[item] = round(model.OBJ(),2)

In [ ]:
s1 = pd.Series(cost.keys(), name="Item_ID")
s2 = pd.Series(cost.values(), name="Cost")
s3 = pd.Series(lot4lot, name = "Lot4Lot")
s = pd.concat([s1,s2], axis=1)
s = pd.concat([s, s3], axis=1)

df_im = pd.merge(df_im, s, on="Item_ID", how="left")

In [ ]:
from google.colab import files

df_im.rename({"Pred": "ForecastedDemand"},axis=1, inplace=True)
df_im.to_csv("Inventory.csv", index=False)
files.download("Inventory.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>